![Erudio logo](../../img/erudio-logo-small.png)

# Itertools

The module `itertools` is a collection of powerful—and carefully designed—functions for performing *iterator algebra*.  That is, these permit *composition* of iterators in sophisticated ways while minimizing concrete instantiation of terms of iterable sequences. In addition to the basic functions in the module itself, the [module documentation](https://docs.python.org/3/library/itertools.html) provides a number of short recipes for additional functions using two or three of the basic module functions in combination. *Be aware that it is easy to get these recipes subtly wrong*. The third-party module `more_itertools` provides additional functions that are likewise designed to avoid common pitfalls and edge cases.

The basic goal of using the building blocks inside `itertools` is to avoid performing computations before they are required, to avoid the memory requirements of large collections, to avoid potentially slow I/O until strictly necessary, and so on. Iterators are lazy sequences rather than realized collections; when combined with functions or recipes in `itertools`, they retain this property.

In [1]:
from itertools import (
    accumulate, 
    count,
    tee,
    takewhile, 
    dropwhile, 
    islice, 
    cycle
)
from math import inf, sqrt, log, isclose

The built-in functions `zip()`, `enumerate()`, `filter()`, `range()`, and `map()` can be consided "honorary itertools" since they can also work with infinite or lazy iterators.  Built-ins like `all()`, `any()`, `sum()`, `min()`, `max()`, and `functools.reduce()` also act on iterables, but all of them, in the general case, need to exhaust the iterator rather than remain lazy.

## Massaging infinite streams of data

Let's create an infinite sequence similar to the primes from the last lesson.  Unlike primes, the Fibonacci sequence requires very little state (i.e. a list of primes found already), only the last two numbers of the sequence.  But like primes, it is an infinite sequence of numbers.

For this implementation, we build in an optional stopping point to more easily get just a finite number of them.

In [2]:
def fibonacci(max=inf, a=1, b=2):
    while a < max:
        yield a
        a, b = b, a+b

In [3]:
for n, f in zip(range(10), fibonacci()):
    print(f"Sequence {n}; Fib={f}")

Sequence 0; Fib=1
Sequence 1; Fib=2
Sequence 2; Fib=3
Sequence 3; Fib=5
Sequence 4; Fib=8
Sequence 5; Fib=13
Sequence 6; Fib=21
Sequence 7; Fib=34
Sequence 8; Fib=55
Sequence 9; Fib=89


In [4]:
for f_ln in map(log, fibonacci(max=60)):
    print(f_ln)

0.0
0.6931471805599453
1.0986122886681098
1.6094379124341003
2.0794415416798357
2.5649493574615367
3.044522437723423
3.5263605246161616
4.007333185232471


In [5]:
map(log, fibonacci(max=60))

## Combining tools

Here is a quick example of combining a few itertools functions. Let's keep a running sum of sum arbitrary iterator. We can create a single lazy iterator to generate both the current number and this sum.

In [6]:
def item_with_total(iterable):
    "Generically transform a stream of numbers into (n, num, running_sum)"
    s, t = tee(iterable)   # unpacking tuples
    yield from zip(count(), t, accumulate(s)) 

This new construct `yield from` is equivalent to:

```python
for n, item, total in zip(count(), t, accumulate(s)):
    yield n, item, total
```

We can apply our new generator function, `item_with_total()` to an arbitrary numeric iterable.

In [7]:
for n, item, total in item_with_total(range(101, 110)):
    print("%3d. Item: %3d; Total: %3d" % (n+1, item, total))

  1. Item: 101; Total: 101
  2. Item: 102; Total: 203
  3. Item: 103; Total: 306
  4. Item: 104; Total: 410
  5. Item: 105; Total: 515
  6. Item: 106; Total: 621
  7. Item: 107; Total: 728
  8. Item: 108; Total: 836
  9. Item: 109; Total: 945


In [8]:
for n, item, total in item_with_total([37, 45, 22, 11, 86, 51]):
    print("%3d. Item: %3d; Total: %3d" % (n+1, item, total))

  1. Item:  37; Total:  37
  2. Item:  45; Total:  82
  3. Item:  22; Total: 104
  4. Item:  11; Total: 115
  5. Item:  86; Total: 201
  6. Item:  51; Total: 252


In [9]:
for n, fib, total in item_with_total(fibonacci(60)):
    print("%3d. Item: %3d; Total: %3d" % (n+1, fib, total))

  1. Item:   1; Total:   1
  2. Item:   2; Total:   3
  3. Item:   3; Total:   6
  4. Item:   5; Total:  11
  5. Item:   8; Total:  19
  6. Item:  13; Total:  32
  7. Item:  21; Total:  53
  8. Item:  34; Total:  87
  9. Item:  55; Total: 142


In [10]:
item_with_total(fibonacci(60))

<generator object item_with_total at 0x0000023C6CC08C80>

## Infinite sequences for convergent sums

Below, we represent a common alternating series whose sum converges to $\ln(2)$.  The sequence by itself is similar to those we have seen for Fibonacci numbers or primes.  But we can use `itertools` to do more with it.

$\ln(2) = \frac{1}{1} - \frac{1}{2} + \frac{1}{3} - \frac{1}{4} + \frac{1}{5} - \cdots $

In [11]:
def ln2_terms():
    sign = 1.
    for denom in count(1):
        yield sign/denom
        sign *= -1

In [12]:
log(2), list(islice(ln2_terms(), 0, 6))

(0.6931471805599453,
 [1.0, -0.5, 0.3333333333333333, -0.25, 0.2, -0.16666666666666666])

We can rephrase this function a bit more concisely and idiomatically (should be slightly faster also).

In [13]:
def ln2_terms():
    for denom, sign in enumerate(cycle([1.,-1.]), 1):
        yield sign/denom

list(islice(ln2_terms(), 0, 6))

[1.0, -0.5, 0.3333333333333333, -0.25, 0.2, -0.16666666666666666]

We might also spell this using (an infinite) generator comprehension.

In [14]:
terms = (sign/denom for (denom, sign) in enumerate(cycle([1.,-1.]), 1))
list(islice(terms, 0, 6))

[1.0, -0.5, 0.3333333333333333, -0.25, 0.2, -0.16666666666666666]

In [15]:
(sign/denom for (denom, sign) in enumerate(cycle([1.,-1.]), 1))

<generator object <genexpr> at 0x0000023C6CC09230>

### Measure convergence

We can use a bit of functional style to define a few convenience functions.

In [16]:
from functools import partial
close_to_ln2 = partial(isclose, log(2), rel_tol=0.01)
close_to_ln2(0.8)

False

In [17]:
close_to_ln2(0.6931)

True

In [18]:
far_from_ln2 = lambda x: not close_to_ln2(x)
far_from_ln2(0.8)

True

In [19]:
running_sum = accumulate(ln2_terms())
for x in takewhile(far_from_ln2, running_sum):
    print(x)

1.0
0.5
0.8333333333333333
0.5833333333333333
0.7833333333333332
0.6166666666666666
0.7595238095238095
0.6345238095238095
0.7456349206349207
0.6456349206349207
0.7365440115440116
0.6532106782106782
0.7301337551337552
0.6587051837051838
0.7253718503718505
0.6628718503718505
0.7216953797836152
0.6661398242280596
0.718771403175428
0.6687714031754279
0.7163904507944756
0.6709359053399302
0.7144141662094954
0.6727474995428288
0.7127474995428288
0.6742859610812904
0.7113229981183273
0.6756087124040416
0.7100914710247312
0.6767581376913979
0.7090162022075269
0.6777662022075269
0.7080692325105572
0.6786574678046748
0.7072288963761034
0.6794511185983256
0.7064781456253526
0.6801623561516684
0.7058033817926941
0.6808033817926941
0.7051936256951331
0.6813841018856093
0.7046399158390977
0.681912643111825
0.7041348653340472
0.6823957348992646
0.7036723306439455
0.6828389973106122
0.7032471605759183
0.6832471605759183
0.7028550037131732
0.683624234482404
0.7024921590107058
0.6839736404921873
0.70215

Actually, that does not converge all that quickly!  The default delta of `1e-09` for `isclose` takes quite a lot of elements before it reaches floating point maximum accuracy.  How many exactly?

In [20]:
def val_not_close_to(target, rel_tol):
    def compare(tup):
        return not isclose(target, tup[1], rel_tol=rel_tol)
    return compare

# Use dropwhile() to exhaust some elements from infinite sequence
close_nums = dropwhile(val_not_close_to(log(2), 1e-6), 
                       enumerate(accumulate(ln2_terms())))
list(islice(close_nums, 0, 3))

[(721346, 0.6931478737072213),
 (721347, 0.6931464874137818),
 (721348, 0.6931478737052995)]

In [21]:
list(islice(close_nums, 0, 3))

[(721349, 0.6931464874157036),
 (721350, 0.6931478737033777),
 (721351, 0.6931464874176254)]

In [22]:
next(close_nums)

(721352, 0.6931478737014559)

In [23]:
%%time
# Let's get something more precise than 1-in-millionth error
# ... it'll take a while!
within_e8 = dropwhile(val_not_close_to(log(2), 1e-8), 
                      enumerate(accumulate(ln2_terms())))
within_e8

CPU times: total: 0 ns
Wall time: 0 ns


In [24]:
%time next(within_e8)

CPU times: total: 31.2 s
Wall time: 32.2 s


(72131429, 0.6931471736284736)

In [25]:
%time next(within_e8)

CPU times: total: 0 ns
Wall time: 0 ns


(72131430, 0.6931471874920554)

Probably time to hit our math textbooks to find a faster convergence if we want a 1-in-a-billionth error.  Clearly the `math` module has a faster method of taking natural logs.

# Exercise

## Description

In this exercise, you should write a function called `merge()` that takes N iterables (each potentially infinite), each of whose iterators yields elements in sorted order (you can rely on that assumption for this exercise).  Your function should produce a new iterator that returns elements in sorted order overall.  If duplicates occur between the iterators (or within one), they should be returned multiple times.

As a motivation for this function, imagine that you have many log files on your system, each of which begins with a timestamp.  You could use this function to order all events from all processes.  Of course, different processes might write different messages at an identical timestamp, and preserving all is relevant.  Moreover, log files can continue to accumulate new records, and this iterator might run forever, monitoring all of them.  In the hypothetical, of course, "sorted order" would simply mean "sorted by first timestamp field."

For this exercise, I provide moderate sized alphabetical wordlists of French, Spanish, and Danish.  Some words occur across multiple languages.  Assuming you open each wordlist as an iterable, your function might produce this:

```python
>>> for word in islice(merge(fr, es, da), 15):
...     print(word)
a
aaronico
ab
ababillar
abaceria
abacero
abaco
abad
abadejo
abadengo
abadernar
abadesa
abadia
abadiato
abaisse
```

**Note**: As you practice, you may use up elements from one or more of the gzip iterators.  Opening them anew will reset the iterables.

**Note 2**: The sort order of words is dependent on your locale, so I have stripped all the words using letters outside the ASCII range in these word lists.  This will assure the same sorting in every locale. Apologies to speakers of those languages who are fond of words removed in the samples.

## Setup

In [26]:
from itertools import *
import gzip

da = gzip.open('wordlist-da.txt.gz', 'rt')
es = gzip.open('wordlist-es.txt.gz', 'rt')
fr = gzip.open('wordlist-fr.txt.gz', 'rt')
langs = [da, es, fr]

def merge(*langs):
    return ['a', 'aaronico', 'ab', 'ababillar', 'tanga']

## Solution

In [27]:
from functools import total_ordering

# Define a value greater than all others
@total_ordering
class Top:
    def __eq__(self, other):
        return False
    def __gt__(self, other):
        return True

# Define a value smaller than all others
@total_ordering
class Floor:
    def __eq__(self, other):
        return False
    def __lt__(self, other):
        return True

top = Top()
floor = Floor()
    
def merge2(a_iter, b_iter):
    topless = lambda x: x is not top and x is not floor
    todo = []
    a, b = floor, floor
    while a is not top or b is not top:
        a = next(a_iter, top)
        todo.append(a)
        for b0 in b_iter:
            todo.append(b0)
            if b0 > a: break

        # Sort and yield the (mostly sorted) temporary accumulation
        # Possible to overshoot by one, so keep last thing in todo
        todo.sort()
        yield from filter(topless, iter(todo[:-1]))
        todo = todo[-1:]

        # Equivalent for a_iter as above with b_iter
        b = next(b_iter, top)
        todo.append(b)
        for a0 in a_iter:
            todo.append(a0)
            if a0 > b: break
        todo.sort()
        yield from filter(topless, iter(todo[:-1]))
        todo = todo[-1:]
                
def merge(*iters):
    all_ = merge2(*iters[:2])
    for it in iters[2:]:
        all_ = merge2(all_, it)
    return all_

## Test Cases

In [28]:
def test_iter():
    from typing import Iterable, Iterator
    assert isinstance(merge(da, es, fr), Iterable)
    assert isinstance(iter(merge(da, es, fr)), Iterator)
    
test_iter()

In [29]:
def test_merge2():
    da = gzip.open('wordlist-da.txt.gz', 'rt')
    fr = gzip.open('wordlist-fr.txt.gz', 'rt')
    merged = merge(da, fr)
    premerged = gzip.open('wordlist-dafr.txt.gz', 'rt')
    # Extra LFs ignored for test
    for a, b in zip_longest(merged, premerged):
        a, b = a.rstrip(), b.rstrip()
        assert a == b, f"Merged {a} does not match {b}"
    
test_merge2()

In [30]:
def test_merge3():
    da = map(str.rstrip, gzip.open('wordlist-da.txt.gz', 'rt'))
    es = map(str.rstrip, gzip.open('wordlist-es.txt.gz', 'rt'))
    fr = map(str.rstrip, gzip.open('wordlist-fr.txt.gz', 'rt'))
    merged = merge(da, es, fr)
    premerged = map(str.rstrip, gzip.open('wordlist-fresda.txt.gz', 'rt'))
    for a, b in zip_longest(merged, premerged):
        assert a == b, f"Merged {a} does not match {b}"
    
test_merge3()

-------------
Materials licensed under [CC BY-NC-ND 4.0](https://creativecommons.org/licenses/by-nc-nd/4.0/) by the authors